# Домашнее задание 2

**Тема “Создание признакового пространства”**

Продолжим обработку данных с Твиттера. 

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.

●	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.

●	Исключим стоп-слова с помощью stop_words='english'.

●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().


2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.

●	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.

●	Исключим стоп-слова с помощью stop_words='english'.

●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

3. Натренируем gensim.models.Word2Vec модель на наших данных.

●	Тренировать будем на токенизированных твитах combine_df['tweet_token']

●	Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.

●	Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.

4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".


5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).

Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.

Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).



In [49]:
import pandas as pd
import numpy as np

import nltk

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os

**1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**

In [50]:
import pickle

with open('combine_df.pkl', 'rb') as f:
    combine_df = pickle.load(f)
    
combine_df.head(3)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"


In [51]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

count_vectorizer = CountVectorizer(max_df=0.9, max_features=1000, stop_words='english')

In [52]:
tweet_stemmed = combine_df['tweet_stemmed'].apply(lambda x: ' '.join(x))

In [53]:
bow_stemmed_tweet = count_vectorizer.fit_transform(tweet_stemmed)

In [54]:
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bow_stemmed_tweet.toarray(), columns = feature_names).head(3)

,abl,absolut,accept,account,act,action,activ,actor,actual,ad,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
tweet_lemmatized = combine_df['tweet_lemmatized'].apply(lambda x: ' '.join(x))

In [56]:
bow_lemmatized_tweet = count_vectorizer.fit_transform(tweet_lemmatized)

In [57]:
pd.DataFrame(bow_lemmatized_tweet.toarray(), columns = feature_names).head(3)

,abl,absolut,accept,account,act,action,activ,actor,actual,ad,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.**

In [58]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=1000, stop_words='english')

In [59]:
bow_tfidf_tweet_stemmed = tfidf_vectorizer.fit_transform(tweet_stemmed)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(bow_tfidf_tweet_stemmed.toarray(), columns = tfidf_feature_names).head(3)

,abl,absolut,accept,account,act,action,activ,actor,actual,ad,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
bow_tfidf_tweet_lemmatized = tfidf_vectorizer.fit_transform(tweet_lemmatized)
pd.DataFrame(bow_tfidf_tweet_lemmatized.toarray(), columns = tfidf_feature_names).head(3)

,abl,absolut,accept,account,act,action,activ,actor,actual,ad,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**3. Натренируем gensim.models.Word2Vec модель на наших данных**

In [61]:
# !pip install gensim

In [62]:
data_tweet_token = combine_df['tweet_token'].tolist()

In [63]:
from gensim.models import Word2Vec

model_w2v = Word2Vec(data_tweet_token, vector_size=200, window=5, min_count=2, sg=1, hs=0, negative=10, seed=34)

model_w2v.train(data_tweet_token, total_examples=len(data_tweet_token), epochs=20)

(9141098, 11726520)

**4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".**

In [64]:
model_w2v.wv.most_similar(positive=['dinner'])

[('bihdaydinner', 0.5594634413719177),
 ('bolognese', 0.5530839562416077),
 ('shawarma', 0.5363757610321045),
 ('waterloo', 0.5313041806221008),
 ('spaghetti', 0.525206446647644),
 ('lamb', 0.522596001625061),
 ('starbuck', 0.520054280757904),
 ('sissy', 0.5185505747795105),
 ('cookout', 0.5148110389709473),
 ('hamburger', 0.5145296454429626)]

In [65]:
model_w2v.wv.most_similar(positive=['trump'])

[('donald', 0.5436174869537354),
 ('unfit', 0.5272117853164673),
 ('donaldtrump', 0.5164005160331726),
 ('dumptrump', 0.5152698159217834),
 ('trumptrain', 0.4998466968536377),
 ('impeachment', 0.4988988935947418),
 ('chopra', 0.49794772267341614),
 ('fuhered', 0.49665477871894836),
 ('conman', 0.49471375346183777),
 ('unfavorability', 0.49403008818626404)]

**5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).**

**Давайте проверим векторное представление любого слова из нашего корпуса, например "food".**

In [66]:
model_w2v.wv['food']

array([ 0.35778326, -0.08280113, -0.25609365, -0.21462314, -0.7645669 ,
       -0.5292339 , -0.06170356,  0.09653826, -0.30805585, -0.16838425,
        0.41810152,  0.38244656, -0.34914726, -0.23346654, -0.4849592 ,
        0.17005771,  0.39897424, -0.2676151 , -0.06216666, -0.02730323,
        0.07680804, -0.12258074, -1.1280146 , -0.32758334,  0.10940449,
        0.5199913 , -0.5925741 ,  0.18249282,  0.75172323,  0.19866101,
        0.5712119 ,  0.17299624, -0.24364601, -0.85439485, -0.4397856 ,
       -1.0188111 ,  0.1454902 ,  0.05237867, -0.05224109, -0.52034944,
       -0.15502183, -0.19468406,  0.524704  ,  0.08106367,  0.0742247 ,
        0.43421626, -0.313001  , -0.709987  , -0.70134616,  0.27784425,
       -0.21587023,  0.19258855,  0.2444823 , -0.08502055,  0.06827107,
       -0.8320021 , -0.29208064, -1.3157969 ,  0.00225663,  0.15279144,
       -0.34020218, -0.18469037, -0.11417313, -0.14224634,  0.27183232,
        0.16958536,  0.2578983 ,  0.03392075,  0.6290588 , -0.68

**6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.**

**Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).**

In [67]:
def tweets2vec(tweets_list, vector_size=200):
    n_words = len(tweets_list)
    vec = np.zeros((1, vector_size))
    if n_words == 0:
        return vec
    for word in tweets_list:
        try:
            vec += model_w2v.wv[word].reshape((1, vector_size))
        except KeyError:
            continue
    return vec / n_words

In [68]:
wordvec_df = combine_df['tweet_token'].apply(tweets2vec)
wordvec_df = np.concatenate(wordvec_df)

In [69]:
wordvec_df.shape

(49159, 200)

In [70]:
with open('wordvec_df.pkl', 'wb') as f:
    pickle.dump(wordvec_df, f)